<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate_Transliter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%bash
rm -rf sample_data
# rm -rf fairseq

In [ ]:
%%bash
rm -rf sample_data
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
git clone https://github.com/pytorch/fairseq.git
cd fairseq
pip install --editable ./
%cd ..
# pip install fairseq


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

In [ ]:
%%bash
# downloading the indic-indic model
wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
unzip indic-en.zip
rm indic-en.zip
wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
unzip m2m.zip
rm m2m.zip

# %cd indicTrans

In [18]:
!pwd
!cp /content/m2m/vocab/bpe_codes.32k.SRC_TGT /content/m2m/vocab/bpe_codes.32k.SRC


/content/indicTrans


In [ ]:
%%bash
apt-get install python-numpy libicu-dev
pip install pyicu pycld2 morfessor 
pip install polyglot


In [ ]:
!polyglot download embeddings2.en transliteration2.ar
!polyglot download transliteration2.kn

In [1]:
# we need to be in indicTrans directory for translation model to load
%pwd
%cd
%cd /content/indicTrans/
%pwd

/root
/content/indicTrans


'/content/indicTrans'

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt  
import easyocr
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from indicTrans.inference.engine import Model
from polyglot.text import Text

class KannadaToHindi :
  def __init__(self):
    self.reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    self.indicm2m_model = Model(expdir='/content/m2m')


  def getTextFromImage(self, image_file):
    reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    result = reader.readtext(image_file, detail = 0)
    return result

  def cleanTransliterText(self, transliter_text):
    blank = ['',"", " ", ' ', '\t']
    s_list = []
    for x in transliter_text:
      if x not in blank:
        s_list.append(x)
    return s_list

  def getTransliterredText(self,text_from_image):
    image_text = str(text_from_image)
    text = Text(image_text)
    transliter_text = text.transliterate("en")
    trans_str = " ".join(self.cleanTransliterText(transliter_text))
    return trans_str

  def TranslateKannadaImageToHindi(self, kannada_img_file):
    text_from_image = self.reader.readtext(kannada_img_file, detail = 0)
    transliterred_result = self.getTransliterredText(text_from_image)
    translated_result = self.indicm2m_model.batch_translate(text_from_image, 'kn', 'hi')
    return translated_result, transliterred_result


In [3]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'
transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

Initializing vocab and bpe
Initializing model for translation


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 21/21 [00:00<00:00, 15545.43it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


hondisi breairi kapadu hni buddivnt snrkshisu mle asn bett ramkrishna gudd kottiruv sriad arisi bitt sthldlli breairi kuvenpu appuge pnditru t mle krishnadevraan asthohdlli iddru nirin mul t
['कृपया इस प्रकार लिखेंः', 'सुरक्षित रखें।', 'बूंद-बूंद', '2 में से', 'बुद्धिमान व्यक्ति', 'इसे सहेजें', 'बारिश', 'सीटों की संख्या', '4 नंबर', 'पर्वत', 'राम कृष्ण', 'पीठ', 'पर्वतमाला', 'सही जवाब चुनने के लिए नीचे दिए गए पते पर लिखेंः', '(शानदार, अप्पू, पंडित,', 'बारिश होती है।', '१. संपर्क', 'श्री कृष्णदेवराय के दरबार में', 'वे मौजूद थे।', 'जल स्रोत', 'संस्कृत']


In [4]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_02.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 9/9 [00:00<00:00, 4696.28it/s]


keutiruv sriad arisi khali bitt sthldlli breairi kuvenpu appuge pnditru mle k shnadevraan asthohdlli pnditru iddru nirin mul mle knndamm n hrke pdyavnnu bredvru kuvenpu alingn pdd arth appuge
['परिसर में दी गई सही जानकारी', 'जवाब चुनें और खाली स्थान पर लिखेंः', '(शानदार, अप्पू, पंडित, संस्कृत, वर्षा)', '1. क.', 'विद्वान भगवान वेंकटेश्वर के दरबार में मौजूद थे।', 'जल स्रोत वर्षा', '3 नहीं', 'उन्होंने कहा कि कन्नड़ कविता की रचना बेहतरीन है।', 'आलिंगन शब्द का अर्थ है कर्ज लेना।']


In [5]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_03.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 16/16 [00:00<00:00, 15760.65it/s]


wakyadlli bisil niru niru aviaaitu mod awag bharwaaitu avi tnidu bharwaaitu vidasagrnondige wad madlu aru vidasagrnondige wad madlu tenali ramkrishna tenali ramkrishna wadkke av grnthvnnu arisikondnu ramkrishna wadkke mhishbndhn enb grnthvnnu arisikondnu knndd knd avudnnu mreabardu knd knndvnnu mreabardu
['एक वाक्य में', 'गर्म पानी का क्या हुआ?', 'गर्मी का पानी भाप बन गया।', '2 में से', 'बादल कब भारी हुआ?', 'उष्मायन वातावरण ठंडा हो गया।', 'कौन विद्यासागर से बहस करने के लिए सहमत हुआ?', 'उत्तरः -', 'तेनाली रामकृष्ण विद्यासागर के साथ बहस करने के लिए सहमत हुए', '4 टेनाली', 'रामकृष्ण ने किस ग्रंथ का चयन किया?', 'उन्होंने उतेनाली रामकृष्ण वाद के लिए तिलकस्थ महिषाबंधन नामक पुस्तक का चयन किया।', 'यह संख्या 5 है', 'हिंदी के प्रति हमें क्या याद रखना चाहिए?', 'कन्नड़ भाषा को न भूलेंः', 'जवाब दीजिएः']


In [6]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 28/28 [00:00<00:00, 10786.23it/s]


prashneglige erdu muru wakyadlli vidasagrn mukh arlitu ramkrishna vidasagrnondige wad madlu karna vidasagrn mukh arlitu mhishbndn endrenu endre elelin kddiglu mhishbndhn endre eamme kttuv hgg ide mhishbndhnd arth kottiruv matnnu aru arige helidru visha modle tilididdre nanu illige bruttle matnnu vidasagr krishnadevraanige helidnu idu nammurin dnkaauv hudugrigu gottu matnnu tenali ramkrishna vidasagrnige helidnu kelgin kvigl pricha madi kuvenpu purna hesru kuppleli venktpp puttpp jnn sthl shivmogg jillea kuppleli irlill
['नीचे दिए गए प्रश्नों के लिए दो/तीन पंक्ति में', 'विद्यासागर का चेहरा क्यों फूला?', 'उथेनाली रामकृष्ण विद्यासागर से बहस करने के लिए सहमत हुई', 'विद्यासागर का चेहरा फूट पड़ा', 'तिलकस्थ महिषाबंदन क्या है?', '0:', 'तिलकाष्ठा', '"" "" "" "बैल की छड़ियां" "" "और" "" "महिषबंधन" "" "यानि बैल की रस्सी" ""', 'तिलकत्था महिषाबंधन का अर्थ', 'ग्यारहवीं का वादा किसने किया?', 'यह विषय पहले से है', 'अगर मैं जानता हूं तो यहां आऊंगा।', 'जवाबः इस', 'विद्यासागर ने कृष्णदेवराय से कहा था -',

In [7]:
# from polyglot.downloader import downloader
# print(downloader.supported_languages_table("transliteration2"))